In [107]:
import tensorflow as tf
import random
import numpy as np
path = 'users.dat'

input_vect_size = 0
with open(path, 'rb') as input_file:
    for line in input_file:
        line = line.strip()
        line = line.split()[1:]
        for item in line:
            if int(item) > input_vect_size:
                input_vect_size = int(item)
            

with open(path, 'rb') as input_file:
    train_set = []
    val_set = []
    for line in input_file:
        line = line.strip()
        line = line.split()
        #num_saved_items = int(line[0])
        #indices = []
        
        full_data = []
        for i in range(1, len(line)):
            full_data.append(int(i))
        training_data = get_training_data(np.array(full_data))
        val_set.append(make_sparse_from_raw_set(full_data, input_vect_size))
        train_set.append(make_sparse_from_raw_set(training_data, input_vect_size))
        
            #indices.append([0, int(line[i]) - 1])
        #values = [1] * num_saved_items
        #shape = [1, input_vect_size]
        #sv = tf.SparseTensor(indices=indices, values=values, shape=shape)

# Need to get the sampled on from test set

In [116]:
import numpy as np
sess = tf.Session()
dv = tf.sparse_tensor_to_dense(sv)
res = sess.run(dv)


In [17]:
a = np.where(res > 0)[1]

In [74]:
def make_sparse_from_raw_set(raw_set, size):
    indices = [[0, i - 1] for i in raw_set]
    values = [1] * len(indices)
    shape = [1, size]
    #return [indices, values, shape]
    return tf.SparseTensor(indices=indices, values=values, shape=shape)

In [89]:
def get_training_data(full_dataset_one_d):
    '''Input: 1-D vector of just the item numbers that are included in
        in the full dataset
       Output: 2-D with the first dimension holding the test set that
               omits 20% of the data, and the second dimension including
               the full set. In this application the validation set is
               the full set.
    '''
    #TODO - Might need to check that training set statisfies a min length
    full_set = list(full_dataset_one_d)
    ind = np.random.randint(1, len(full_dataset_one_d), len(full_dataset_one_d))
    train_set_ind = np.where(ind <= 0.8 * len(full_dataset_one_d))[0]
    train_set = full_set[train_set_ind]
    
    return train_set

In [59]:
b, c = get_validation_set(a)

In [60]:
b

array([ 1630,  2316,  2525,  2845,  2930,  3170,  3296,  3331,  3403,
        3594,  3769,  3949,  4661,  4870,  4888,  5113,  5323,  5324,
        5613,  5990,  6102,  6873,  6967,  7105,  7800,  7866,  8902,
        9906, 10007, 10203, 10287, 10507, 10587, 11008, 11104, 11225,
       11852, 11918, 12683, 12715, 12801, 12803, 12804, 12830, 12915,
       13171, 13362, 13922, 13923, 14661, 14675, 14850, 15025, 15164,
       15190, 15281, 15299, 15335, 15832, 15893, 16162, 16423])

In [61]:
len_b = b.shape[0]
len_a = a.shape[0]
print float(len_b) / len_a

0.885714285714


In [75]:
out = make_sparse_vect_from_raw_set(b,input_vect_size)

In [76]:
out

In [120]:
ex = lambda i: [i]
ex(1)


[1]

In [139]:
a = tf.Variable([1,1,1])
b = tf.Variable([2,2,2])
#tf.pack([a, b])
#tf.Variable([[1,2]])
res = tf.matmul(tf.Variable([[1,2]]), [a, b])

In [140]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(res)

array([[5, 5, 5]], dtype=int32)

In [2]:
W_init_val = [tf.constant(range(i, i+2), dtype=tf.float32) for i in range(1, 7, 2)]
W_prime_init_val = [tf.constant(range(i, i+2), dtype=tf.float32) for i in range(7, 13, 2)]
b_prime_init_val = [tf.constant(2.0, dtype=tf.float32) for _ in xrange(3)]

In [89]:
import tensorflow as tf
sess = tf.Session()
optimizer = tf.train.GradientDescentOptimizer(0.01)

y_0 = [[0, 1, 1]]
y = tf.placeholder(tf.float32, [None, 3])

# Test Gradients
weight = dict()
weight['W'] = [tf.Variable(seq) for seq in W_init_val]
weight['W_prime'] = [tf.Variable(seq) for seq in W_prime_init_val]
weight['b'] = tf.Variable(tf.constant([[1.0, 1.0]], dtype=tf.float32))
weight['b_prime'] = [tf.Variable(seq) for seq in b_prime_init_val]
z = tf.add(tf.matmul(y, weight['W']), weight['b'])
y_hat = tf.add(tf.matmul(z, tf.transpose(weight['W_prime'])), weight['b_prime'])

loss_0 = tf.nn.l2_loss(tf.sub(y, y_hat))

sess.run(tf.initialize_all_variables())
W_var_before = [sess.run(var) for var in weight['W']]
W_prime_var_before = [sess.run(var) for var in weight['W_prime']]

#Gradient
#dw_0 = tf.gradients(loss_0, [weight['W'][0]])[0]
#dw_prime_0 = tf. gradients(loss_0, [weight['W_prime'][0]])[0]
dw_prime_0 = optimizer.minimize(loss_0, var_list=[weight['W_prime'][0]])
#dw_1 = tf.gradients(loss_0, [weight['W'][1]])[0]
#dw_prime_1 = tf. gradients(loss_0, [weight['W_prime'][1]])[0]
sess.run(dw_prime_0, feed_dict={y: y_0})
W_var = [sess.run(var) for var in weight['W']]
W_prime_var = [sess.run(var) for var in weight['W_prime']]
#z_out, y_hat_out, loss, grad_w0, grad_w_prime0, grad_w1, grad_w_prime1 = sess.run((z, y_hat, loss_0, dw_0, dw_prime_0, dw_1, dw_prime_1), feed_dict={y: y_0})

normalized_weights = [tf.nn.l2_normalize(var, 0) for var in weight.iteritems()]
w_sum = tf.reduce_sum(normalized_weights)

reg_sum = sess.run(w_sum)


TypeError: l2_normalize() takes at least 2 arguments (1 given)

In [93]:
type(W_prime_var[0])

numpy.ndarray

In [18]:
import tensorflow as tf
sess = tf.Session()
optimizer = tf.train.GradientDescentOptimizer(0.01)

y_0 = [[1, 0, 1]]
y = tf.placeholder(tf.float32, [None, 3])

# Test Gradients
weight = dict()
weight['W'] = [tf.Variable(seq) for seq in W_init_val]
weight['W_prime'] = [tf.Variable(seq) for seq in W_prime_init_val]
weight['b'] = tf.Variable(tf.constant([[1.0, 1.0]], dtype=tf.float32))
weight['b_prime'] = [tf.Variable(seq) for seq in b_prime_init_val]
z = tf.add(tf.matmul(y, weight['W']), weight['b'])
y_hat = tf.add(tf.matmul(z, tf.transpose(weight['W_prime'])), weight['b_prime'])

loss_0 = tf.nn.l2_loss(tf.sub(y, y_hat))
#loss_1 = tf.nn.l2_loss(tf.sub(y, y_hat))
#cost = 0.5 * tf.reduce_sum(tf.nn.l2_loss(tf.sub(y, y_hat)))
sess.run(tf.initialize_all_variables())
W_var_before = [sess.run(var) for var in weight['W']]
W_prime_var_before = [sess.run(var) for var in weight['W_prime']]

#Gradient
#dw_0 = tf.gradients(loss_0, [weight['W'][0]])[0]
#dw_prime_0 = tf. gradients(loss_0, [weight['W_prime'][0]])[0]
dw_prime_0 = optimizer.minimize(loss_0, var_list=[weight['W_prime'][0]])
#dw_1 = tf.gradients(loss_0, [weight['W'][1]])[0]
#dw_prime_1 = tf. gradients(loss_0, [weight['W_prime'][1]])[0]
#sess.run(dw_prime_0, feed_dict={y: y_0})
#W_var = [sess.run(var) for var in weight['W']]
#W_prime_var = [sess.run(var) for var in weight['W_prime']]
#z_out, y_hat_out, loss, grad_w0, grad_w_prime0, grad_w1, grad_w_prime1 = sess.run((z, y_hat, loss_0, dw_0, dw_prime_0, dw_1, dw_prime_1), feed_dict={y: y_0})

#normalized_weights = [tf.nn.l2_normalize(var, 0) for var in weight.iteritems()]
#w_sum = tf.reduce_sum(normalized_weights)

#reg_sum = sess.run(w_sum)


In [40]:
l = 0.1
loss = tf.nn.l2_loss(tf.sub(y, y_hat))

dloss_d = lambda var: optimizer.compute_gradients(loss, var_list=[var])[0][0]
dCost_d = lambda var: dloss_d(var) - tf.mul(l, var)

grad_op = lambda var: optimizer.apply_gradients([[dCost_d(var), var]])
W_var_before = [sess.run(var) for var in weight['W']]
#cost = lambda var: loss - tf.mul(l, var)
#cost = 0.5 * tf.reduce_sum([tf.nn.l2_loss(vect) for vect in tf.sub(z_0, z)])
#cost = 0.5 * tf.reduce_sum(tf.nn.l2_loss(tf.sub(z_0, z)))
sess.run(grad_op(weight['W'][0]), feed_dict={y: y_0})
W_var = [sess.run(var) for var in weight['W']]

In [41]:
W_var_before

[array([ 1.,  2.], dtype=float32),
 array([ 3.,  4.], dtype=float32),
 array([ 5.,  6.], dtype=float32)]

In [42]:
W_var

[array([-41.9489975, -45.5779953], dtype=float32),
 array([ 3.,  4.], dtype=float32),
 array([ 5.,  6.], dtype=float32)]